<a href="https://colab.research.google.com/github/iveliz/ModelosClasificacionMachineLearning/blob/develop/ProyectoClasificacion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !pip install pandas
# !pip install sklearn

In [ ]:
# Libs de manipulacion de datos
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
# Libs de preprocesamineto y procesamiento
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
#----- Para mostrar graficos y encodings
from tabulate import tabulate
# Matriz de confucion

In [ ]:
# chunks = [(0, 1133)] # 
chunks = [(2247, 8354)] #  Se lee desde la fila 2247 porque desde esta fila se cuenta con los 2 features adicionales (imps)
df = pd.DataFrame()
for start, end  in chunks:  
  chunk = pd.read_csv('/content/drive/MyDrive/ML-LASTLONG/dataset_fuel_consumption_2016_2022.csv', 
              header=start, nrows=end, usecols=range(0,15)) #15
  df = df.append(chunk)
df = df.dropna() # Hacemos un drop de los que tengan valores (NaN) en tre sus celdas 
# df = df.drop(index=0).reset_index(drop=True)

<ipython-input-4-e35b00fc101f>:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(chunk)


In [ ]:
# df = df.dropna() # Hacemos un drop de los que tengan valores (NaN) en tre sus celdas 
# df.head()

In [ ]:
# Verficamos que no existan mas valores NAN 
null_rows = df.isnull().any(axis=1)
rows_with_nulls = df[null_rows]
rows_with_nulls
# df.isna().sum()

# df = df.dropna() # NaN = Not a number

,Model,Make,Model.1,Vehicle Class,Engine Size,Cylinders,Transmission,Fuel,Fuel Consumption,Unnamed: 9,Unnamed: 10,Unnamed: 11,CO2 Emissions,CO2,Smog


# **Renombramiento de features**
Se renombran los features con los especificados en el proyecto

In [ ]:
# Creamos un dict con los nuevos nombres para los respectivos nombres ya capturados

df.rename(columns = {'Model': 'MODEL YEAR', 
                     'Make': 'MAKE', 'Model.1': 'MODEL', 'Vehicle Class': 'VEHICLE CLASS','FUEL': 'FUEL TYPE', 
                     'Engine Size':'ENGINE SIZE (L)',                     
                     'Transmission': 'TRANSMISSION',
                     'Fuel':'FUEL TYPE',
                     'Fuel Consumption':'FUEL CONSUMPTION CITY (L/100 km)', 
                     'Unnamed: 9':'FUEL CONSUMPTION HWY (L/100 km)', 
                     'Unnamed: 10':'FUEL CONSUMPTION COMB (L/100 km)',  
                     'Unnamed: 11':'FUEL CONSUMPTION COMB (mpg)',   
                     'CO2 Emissions':'CO2 EMISSIONS (g/km)',
                     'CO2':'CO2 Rating',
                     'Smog':'Smog Rating',
                     }, 
                     inplace=True)

df = df.astype({'MODEL YEAR': int, 'ENGINE SIZE (L)': float, 'Cylinders': int, 
                'FUEL CONSUMPTION CITY (L/100 km)': float, 
                'FUEL CONSUMPTION HWY (L/100 km)': float,
                'FUEL CONSUMPTION COMB (L/100 km)': float,
                'FUEL CONSUMPTION COMB (mpg)': int, 'CO2 EMISSIONS (g/km)': int, 'CO2 Rating':int, 'Smog Rating': int
                })

In [ ]:
# df.head()
# df.isna().sum()
# df.tail()
# print("size:",df.size)
# print("size:",df.shape)
# df['Unnamed: 10 COMB (L/100 km)']
# df['CYLINDERS'].value_counts()
[i for i in df.columns]

['MODEL YEAR',
 'MAKE',
 'MODEL',
 'VEHICLE CLASS',
 'ENGINE SIZE (L)',
 'Cylinders',
 'TRANSMISSION',
 'FUEL TYPE',
 'FUEL CONSUMPTION CITY (L/100 km)',
 'FUEL CONSUMPTION HWY (L/100 km)',
 'FUEL CONSUMPTION COMB (L/100 km)',
 'FUEL CONSUMPTION COMB (mpg)',
 'CO2 EMISSIONS (g/km)',
 'CO2 Rating',
 'Smog Rating']

# Exploracion de datos

In [ ]:
plt.figure(figsize=(8, 15))  # set figure size
df['MAKE'].value_counts().plot.barh()
plt.xlabel('Recuento')
plt.ylabel('Fabricante')
plt.show()

In [ ]:
# df['Smog'].value_counts()
# sns.countplot(df['Smog Rating'])
df.isna().sum()

MODEL YEAR                          0
MAKE                                0
MODEL                               0
VEHICLE CLASS                       0
ENGINE SIZE (L)                     0
Cylinders                           0
TRANSMISSION                        0
FUEL TYPE                           0
FUEL CONSUMPTION CITY (L/100 km)    0
FUEL CONSUMPTION HWY (L/100 km)     0
FUEL CONSUMPTION COMB (L/100 km)    0
FUEL CONSUMPTION COMB (mpg)         0
CO2 EMISSIONS (g/km)                0
CO2 Rating                          0
Smog Rating                         0
dtype: int64

In [ ]:
sns.set_style('darkgrid')
# create the countplot
ax = sns.countplot(x=df['FUEL TYPE'])
# set the x-axis label
ax.set_xlabel('FUEL TYPE')
# rotate the x-axis labels by 45 degrees
plt.xticks(rotation=45)
# display the plot
plt.show()

In [ ]:
df.columns

Index(['MODEL YEAR', 'MAKE', 'MODEL', 'VEHICLE CLASS', 'ENGINE SIZE (L)',
       'Cylinders', 'TRANSMISSION', 'FUEL TYPE',
       'FUEL CONSUMPTION CITY (L/100 km)', 'FUEL CONSUMPTION HWY (L/100 km)',
       'FUEL CONSUMPTION COMB (L/100 km)', 'FUEL CONSUMPTION COMB (mpg)',
       'CO2 EMISSIONS (g/km)', 'CO2 Rating', 'Smog Rating'],
      dtype='object')

In [ ]:
# df_fueltype = df.groupby(['FUEL TYPE'],as_index=False).mean()
# df_fueltype

## **Codificación de datos tipo STR(Cadena)**
Se ha de codificar (Porque la computadora y los modelos matemáticos sólo pueden trabajar con datos numéricos, por lo que es necesario codificar los tipos de datos de cadena en números): 
Para maker seria: 

| MAKE | EncodedLabel |
| --- | --- | 
| ACURA | 0 |
| ALFA ROMEO | 1 |
| ASTON MARTIN | 2 |

Para MODEL:

| MODEL.1 | EncodedLabel |
| --- | --- | 
| ILX | 355 |
| ILX HYBRID | 356 |
|  MDX SH-AWD | 422 |




In [ ]:
# df.columns

Index(['MODEL YEAR', 'MAKE', 'MODEL', 'VEHICLE CLASS', 'ENGINE SIZE (L)',
       'Cylinders', 'TRANSMISSION', 'FUEL TYPE',
       'FUEL CONSUMPTION CITY (L/100 km)', 'FUEL CONSUMPTION HWY (L/100 km)',
       'FUEL CONSUMPTION COMB (L/100 km)', 'FUEL CONSUMPTION COMB (mpg)',
       'CO2 EMISSIONS (g/km)', 'CO2 Rating', 'Smog Rating', 'MODEL.1'],
      dtype='object')

In [ ]:
label_encoder = preprocessing.LabelEncoder()
# PRE-TREATEMENT LABELING ----------- [IMP 1]
make_values = label_encoder.fit_transform(df['MAKE'])
model_values = label_encoder.fit_transform(df['MODEL'])
vehicleClass_values = label_encoder.fit_transform(df['VEHICLE CLASS'])
transmition_values = label_encoder.fit_transform(df['TRANSMISSION'])  #TRANSMISSION 
fuel_type = label_encoder.fit_transform(df['FUEL TYPE'])   

# create a dictionary mapping original names to encoded labels
make_to_label = dict(zip(df['MAKE'], make_values))
model_to_label = dict(zip(df['MODEL'], model_values))
vehicle_to_label = dict(zip(df['VEHICLE CLASS'], vehicleClass_values))
transmition_to_label = dict(zip(df['TRANSMISSION'], transmition_values))
fuel_to_label = dict(zip(df['FUEL TYPE'], fuel_type))

# # create a table of original names and encoded labels
# table1 = {'MAKE': list(make_to_label.keys()), 'EncodedLabel': list(make_to_label.values())}
# table2 = {'MODEL': list(model_to_label.keys()), 'EncodedLabel': list(model_to_label.values())}
# table3 = {'VEHICLE CLASS': list(vehicle_to_label.keys()), 'EncodedLabel': list(vehicle_to_label.values())}
# table4 = {'TRANSMISSION': list(transmition_to_label.keys()), 'EncodedLabel': list(transmition_to_label.values())}
# table5 = {'FUEL TYPE': list(fuel_to_label.keys()), 'EncodedLabel': list(fuel_to_label.values())}

# TREATEMENT -----------[IMP 2]
df['MAKE'] = label_encoder.fit_transform(df['MAKE']) # Nissan ->  2
df['MODEL'] = label_encoder.fit_transform(df['MODEL']) # C -> 3
df['VEHICLE CLASS'] = label_encoder.fit_transform(df['VEHICLE CLASS'])
df['TRANSMISSION'] = label_encoder.fit_transform(df['TRANSMISSION'])  #TRANSMISSION 
df['FUEL TYPE'] = label_encoder.fit_transform(df['FUEL TYPE'])    

In [ ]:
from sklearn import tree
clf = tree.DecisionTreeClassifier() #1200 primeras lineas (2) > 83 - 85 

# ----------- TARGET -------: CO2 EMISSIONS  (g/km)
X = df.drop(columns = ['CO2 EMISSIONS (g/km)'])
y = df['CO2 EMISSIONS (g/km)']  # Nissan 5 D 15 45 5 5 dd ->  45
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20) # 20% 20
model = clf.fit(X_train , y_train)

In [ ]:
# print(tabulate(table2, headers=[list(table2.keys())[0], list(table2.keys())[1]], tablefmt="grid"))
#              table(n)              table(n)                table(n)

In [ ]:
predictions = model.predict(X_test) # Datos de prueba
score = accuracy_score(y_test, predictions) # 50 <- # 50
score  # 
# len(y_test)
# print(y_test.item[0], predictions.item[0] )

0.5978705978705978

# Recuento de datos 

In [ ]:
len(X_train) + len(y_test) # 4880 para entrenamiento y 1221 para testing  
 

6101

# Metricas

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

# assume y_true and y_pred are the true and predicted labels
cm = confusion_matrix(y_test, predictions)
num_correct = sum(np.diag(cm))
num_incorrect = cm.sum() - num_correct
print(f"Number of correct predictions: {num_correct}")
print(f"Number of incorrect predictions: {num_incorrect}")

In [ ]:
# from sklearn.metrics import precision_score
# precision_score(y_test, predictions, average='macro')
# from sklearn import metrics
# fpr, tpr, thresholds = metrics.roc_curve(y_test, predictions, pos_label=30)
# metrics.auc(fpr, tpr)
from sklearn.metrics import hamming_loss
# average_precision_score(y_test, predictions)
hamming_loss(y_test, predictions)

from sklearn.metrics import jaccard_score
jaccard_score(y_test, predictions, average="micro")

0.48630553864881315

# Pruebas con nuevos datos

In [ ]:
import random
random_make, random_make_value = random.choice(list(make_to_label.items()))
random_model, random_model_value = random.choice(list(model_to_label.items()))
random_vehicle, random_vehicle_value = random.choice(list(vehicle_to_label.items()))
random_transmition, random_transmition_value = random.choice(list(transmition_to_label.items()))
random_fuel, random_fuel_value  = random.choice(list(fuel_to_label.items()))

# Print the randomly picked items 
print(f"Make:          {random_make:{20}}        Value: {random_make_value}")
print(f"Model:         {random_model:{21}}       Value: {random_model_value}")
print(f"Vehicle Class: {random_vehicle:{25}}     Value: {random_vehicle_value}")
print(f"Transmission:  {random_transmition:{25}} Value: {random_transmition_value}")
print(f"Fuel Type:     {random_fuel:{20}}        Value: {random_fuel_value}")

Make:          LEXUS                       Value: 42
Model:         TRANSIT CONNECT VAN FFV       Value: 1739
Vehicle Class: STATION WAGON - MID-SIZE      Value: 15
Transmission:  A10                       Value: 0
Fuel Type:     E                           Value: 1


In [ ]:
new_data = {'MODEL YEAR': 2018,
 'MAKE': random_make_value,
 'MODEL': random_model_value,
 'VEHICLE CLASS': random_vehicle_value,
 'ENGINE SIZE (L)': 2,
 'Cylinders': 6,
 'TRANSMISSION': random_transmition_value,
 'FUEL TYPE': random_fuel_value,
 'FUEL CONSUMPTION CITY (L/100 km)': 9.7,
 'FUEL CONSUMPTION HWY (L/100 km)': 6.9,
 'FUEL CONSUMPTION COMB (L/100 km)':10.3,
 'FUEL CONSUMPTION COMB (mpg)': 34,
 'CO2 Rating': 6,
 'Smog Rating': 1}

new_df = pd.DataFrame([new_data])
# new_data = [[2019, 4, 200, 4, 1.8, 4, 3, 2, 8.0, 6.2, 7.0, 32, 7, 4]]
prediction = model.predict(new_df)
print(f"Prediccion hecha para el DF: {new_df}---> \n {prediction}")

array([238])

In [ ]:
new_df

,MODEL YEAR,MAKE,MODEL,VEHICLE CLASS,ENGINE SIZE (L),Cylinders,TRANSMISSION,FUEL TYPE,FUEL CONSUMPTION CITY (L/100 km),FUEL CONSUMPTION HWY (L/100 km),FUEL CONSUMPTION COMB (L/100 km),FUEL CONSUMPTION COMB (mpg),CO2 Rating,Smog Rating
0,2018,42,1739,15,2,6,0,1,9.7,6.9,10.3,34,6,1


# Exploracion de resultados del modelo Desicion Tree Classifier

In [ ]:
predicted = []
incorrect = []
for i in range(len(X_test)):
  # Get the attributes for this example
  example = X_test.iloc[i].values
  # Predict the label for this example
  y_pred = model.predict([example])
  # Append the predicted label to the predicted list
  
  # Get the true label for this example
  y_true = y_test.iloc[i]
  
  # Compare the predicted label to the true label and append to the correct list if they match
  if y_pred == y_true:
    predicted.append({'x':example ,'y':y_pred , 'expected': y_true})  
    continue
  else:
    incorrect.append({'x':example ,'y':y_pred , 'expected': y_true})
    # reversed = []
    # j = 0
    # indexes = [55, 1, 2, 3, 55, 55, 6, 7, 55, 55, 55, 55, 55, 55]
    # for i, x in enumerate(example):      
    #   if x == 1:  
    #     key = {v: k for k, v in make_to_label.items()}[int(example[i])]  
    #     print(key)      
    #     reversed[x]=key  
    #     i += 1    
    #   else: 
    #     reversed.append(x)

In [ ]:
# len(X_test)
print(f'De los {len(X_test)} datos, se predijeron {len(predicted)} correctamente  y {len(incorrect)} incorrectamente')


De los 1221 datos, se predijeron 759 correctamente  y 462 incorrectamente


In [ ]:
import random
j = 0
indexes = [55, 1, 2, 3, 55, 55, 6, 7, 55, 55, 55, 55, 55, 55]
incorrect = random.randint(0,len(predicted)-1)
for i, x in enumerate(predicted[incorrect]['x']):    
  if i == 1:
    key = {v: k for k, v in make_to_label.items()}[int(x)]
    print(key)
  elif i == 2:
    key = {v: k for k, v in model_to_label.items()}[int(x)]
    print(key)
      
  elif i == 3:
    key = {v: k for k, v in vehicle_to_label.items()}[int(x)]
    print(key)
  elif i == 6:
    key = {v: k for k, v in transmition_to_label.items()}[int(x)]
    print(key)
  elif i == 7:
    key = {v: k for k, v in fuel_to_label.items()}[int(x)]
    print(key)
  else:
    print(x)
  j += 1  

new_dict = {key: value for key, value in predicted[incorrect].items() if key != 'x'}
new_dict

2019.0
Porsche
Panamera 4 Executive
Full-size
3.0
6.0
AM8
Z
12.4
9.0
10.9
26.0
4.0
5.0


{'y': array([253]), 'expected': 253}

In [ ]:
{'MODEL YEAR': 2019,
 'MAKE': random_make_value,
 'MODEL': random_model_value,
 'VEHICLE CLASS': random_vehicle_value,
 'ENGINE SIZE (L)': 2.0,
 'Cylinders': 4,
 'TRANSMISSION': random_transmition_value,
 'FUEL TYPE': random_fuel_value,
 'FUEL CONSUMPTION CITY (L/100 km)': 10.8,
 'FUEL CONSUMPTION HWY (L/100 km)': 7.6,
 'FUEL CONSUMPTION COMB (L/100 km)': 9.3,
 'FUEL CONSUMPTION COMB (mpg)': 30,
 'CO2 Rating': 5,
 'Smog Rating': 5}

In [ ]:
# matrix = confusion_matrix(y_test, predictions)
# disp = ConfusionMatrixDisplay(matrix )
# disp.plot()
# plt.show()

## Model con **KNeighborsClassifier**
Support Vector Machines

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
neigh = KNeighborsClassifier()
neigh.fit(X_train , y_train)  
predictionsKN = neigh.predict(X_test)

In [ ]:
scoreKN = accuracy_score(y_test, predictionsKN)
print(scoreKN)

0.20393120393120392


In [ ]:
from sklearn.svm import SVC
clf_SVC = SVC()
clf_SVC.fit(X_train, y_train)
predictions_SVC = clf_SVC.predict(X_test)
clf_SVC.score(X_test, predictions_SVC)

1.0

# Confussion Matrix

In [ ]:
from sklearn.metrics import classification_report
cm = confusion_matrix(y_test, predictions_SVC, labels=clf_SVC.classes_)
disp = ConfusionMatrixDisplay(confusion_matrix=cm,
                 display_labels=clf_SVC.classes_)
# disp.plot()
# plt.show()
# print(classification_report(y_test, predictions_SVC))

In [ ]:
scoreSVC = accuracy_score(y_test, predictions_SVC)
scoreSVC
# clf_SVC.score(X_test, predictions_SVC)
scoreSVC*100

1.4742014742014742

# Modelo con RandomForestClassifier


In [ ]:
from sklearn.ensemble import RandomForestClassifier
clf_Rand = RandomForestClassifier(random_state=0) # 
clf_Rand.fit(X_train, y_train)
predictions_Rand = clf_Rand.predict(X_test)
scoreRand = accuracy_score(y_test, predictions_Rand)
scoreRand*100 # 

68.38656838656838